In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np
import keras
from keras import layers


In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

--2021-02-04 07:47:37--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 142.250.98.128, 172.217.193.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2021-02-04 07:47:37 (86.8 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [3]:
total_words

3211

In [4]:
len(corpus)

2159

In [5]:
corpus[:10]

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,']

In [4]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [5]:
predictors.shape

(15462, 10)

In [6]:
# max_sequence_len demonstrates the max number of words in sentence
max_sequence_len-1

10

In [7]:

model = keras.models.Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(layers.Dense(total_words, activation='relu'))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.LSTM(128))
model.add(layers.Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
dense (Dense)                (None, 10, 3211)          324311    
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           1710080   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              414219    
Total params: 3,032,878
Trainable params: 3,032,878
Non-trainable params: 0
______________________________________________

In [9]:
# where does the 484 in each epoch come from?
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 5s 11ms/step - loss: 6.8831 - accuracy: 0.0204
Epoch 2/100
484/484 [==============================] - 5s 10ms/step - loss: 6.5072 - accuracy: 0.0237
Epoch 3/100
484/484 [==============================] - 5s 10ms/step - loss: 6.4003 - accuracy: 0.0330
Epoch 4/100
484/484 [==============================] - 6s 11ms/step - loss: 6.3095 - accuracy: 0.0360
Epoch 5/100
484/484 [==============================] - 5s 10ms/step - loss: 6.2137 - accuracy: 0.0395
Epoch 6/100
484/484 [==============================] - 5s 11ms/step - loss: 6.1019 - accuracy: 0.0408
Epoch 7/100
484/484 [==============================] - 5s 11ms/step - loss: 5.9797 - accuracy: 0.0480
Epoch 8/100
484/484 [==============================] - 5s 11ms/step - loss: 5.8558 - accuracy: 0.0487
Epoch 9/100
484/484 [==============================] - 5s 10ms/step - loss: 5.7305 - accuracy: 0.0548
Epoch 10/100
484/484 [==============================] - 6s 12ms/step - loss: 5.603

484/484 [==============================] - 5s 10ms/step - loss: 0.9324 - accuracy: 0.8018
Epoch 86/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9106 - accuracy: 0.8082
Epoch 87/100
484/484 [==============================] - 5s 10ms/step - loss: 0.8974 - accuracy: 0.8066
Epoch 88/100
484/484 [==============================] - 6s 11ms/step - loss: 0.8930 - accuracy: 0.8090
Epoch 89/100
484/484 [==============================] - 5s 10ms/step - loss: 0.8798 - accuracy: 0.8113
Epoch 90/100
484/484 [==============================] - 5s 11ms/step - loss: 0.8614 - accuracy: 0.8149
Epoch 91/100
484/484 [==============================] - 5s 10ms/step - loss: 0.8362 - accuracy: 0.8197
Epoch 92/100
484/484 [==============================] - 5s 11ms/step - loss: 0.8232 - accuracy: 0.8205
Epoch 93/100
484/484 [==============================] - 5s 10ms/step - loss: 0.8176 - accuracy: 0.8214
Epoch 94/100
484/484 [==============================] - 5s 10ms/step - loss: 0.8002 - 

In [ ]:
#couldn't get it to work

import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()


In [ ]:
#couldn't get it to work
#outcome of the seed text without - with learning rate chosen for Adam optimizer 
seed_text = "Hell is empty"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

Trained the model for 100 epochs and that made a difference.

The accuracy jumped from 50% to 83% Also the loss went down from 2% to 0.7% Whoa that's great. Now what? Maybe we can optimize the model building and get to a computationally less intensive one?